## Compute simplicial homology

In [214]:
import numpy as np
import string






def kernel(A, tol=1e-5):
    """
    Return a matrix whose column space is the kernel of A.
    The tol parameter is the threshold below which a singular value is considered to be zero.
    Taken from: https://github.com/kb1dds/simplicialHomology/blob/master/simplicialHomology.py
    """
    _, s, vh = np.linalg.svd(A)
    singular = np.zeros(vh.shape[0], dtype=float)
    singular[:s.size] = s
    null_space = np.compress(singular <= tol, vh, axis=0)
    return null_space.T


def cokernel(A, tol=1e-5):
    """
    Return a matrix whose column space is the cokernel of A.
    The tol parameter is the threshold below which a singular value is considered to be zero.
    Taken from: https://github.com/kb1dds/simplicialHomology/blob/master/simplicialHomology.py
    """
    u, s, _ = np.linalg.svd(A)
    singular = np.zeros(u.shape[1], dtype=float)
    singular[:s.size] = s
    return np.compress(singular <= tol, u, axis=1)


def get_faces(lst):
    """Compute all the possible faces by iteratively deleting vertices"""
    return [lst[:i] + lst[i+1:] for i in range(len(lst))]


def get_coeff(simplex, faces):
    """
    If simplex is not in the list of faces, return 0.
    If it is, return index parity.
    """
    if simplex in faces:
        idx = faces.index(simplex)
        return 1 if idx%2==0 else -1
    else:
        return 0


def boundary(complex):
    """
    Given an abstract simplicial complex specified as a list of lists of vertices, return a
    list of boundary operators in matrix form.
    """
    # Get maximal simplex dimension
    maxdim = len(max(complex, key=len))
    # Group simplices by (ascending) dimension and sort them lexicographically
    simplices = [sorted([spx for spx in complex if len(spx)==i]) for i in range(1,maxdim+1)]

    # Iterate over consecutive groups (dim k and k+1)
    bnd = []
    for spx_k, spx_kp1 in zip(simplices, simplices[1:]):
        mtx = []
        for sigma in spx_kp1:
            faces = get_faces(sigma)
            mtx.append([get_coeff(spx, faces) for spx in spx_k])
        bnd.append(np.array(mtx).T)

    return bnd


def homology(boundary_ops, tol=1e-5):
    """
    Given a list of boundary operators, return a list of matrices whose columns
    span the homology spaces. 
    """
    # Insert zero maps
    mm = boundary_ops[-1].shape[1]
    nn = boundary_ops[0].shape[0]

    boundary_ops.insert(0, np.ones(shape=(0, nn)))
    boundary_ops.append(np.ones(shape=(mm, 0)))

    H = []
    for del_k, del_kp1 in zip(boundary_ops, boundary_ops[1:]):
        # Compute a basis for the kernel of the next map
        kappa = kernel(del_k, tol)
        # The chain complex induces a map m from previous space to the kernel of next map
        # Solve d_{k} = kappa \circ m for m
        psi, _, _, _ = np.linalg.lstsq(kappa, del_kp1, rcond=None)
        # The cokernel of m is precisely those elements of the kernel of the next map
        # that are not in the image of m (or d_k for that matter), that's homology
        ksi = cokernel(psi, tol)
        # Express a basis for the homology thought of as a subspace of C_k
        # using composition of maps
        H.append(np.dot(kappa, ksi))

    return H


def betti(H):
    """Compute the dimensions of each homology space output by the homology() function"""
    return [basis.shape[1] for basis in H]


def main():
    with open('./complexes.json') as data_file:
        data = json.load(data_file)

    for attr, obj in data.items():
        print(f'[{attr}]')
        print(obj['description'])
        bnd = boundary(obj['complex'])
        H = homology(bnd)
        b = betti(H)
        print(f'Betti numbers: {b}')
def compute_laplacians(boundary_ops):
    """Takes the list of boundary operations and computes the laplacian operators"""
    laplacians=[]
    if boundary_ops==[]:
        return np.array([0])
    else:
        dim=len(boundary_ops)
        for i in range(dim+1):
            if i ==0:
                D=boundary_ops[i]@boundary_ops[i].T
                laplacians.append(D)
            elif i==dim:
                D=boundary_ops[i-1].T@boundary_ops[i-1]
                laplacians.append(D)
            else:
                D=(boundary_ops[i-1].T)@boundary_ops[i-1]+boundary_ops[i]@boundary_ops[i].T
                laplacians.append(D)
    return laplacians

def compute_harmonics(laplacians):
    """Takes a list of Laplacian matrices and computes the dimension of harmonics"""
    return [kernel(M).shape[1] for M in laplacians]

def make_polygon(n):
    """Makes a polygon with n sides"""
    """Create a list of uppercase letters"""
    uppercase_letters = list(string.ascii_uppercase)
    if n>len(uppercase_letters):
        return "Not enough letters"
    else:
        ans=dict()
        ans["description"]=f"A polygon with {n} sides "
        ans["complex"]=uppercase_letters[:n]
        for i in range(n):
            ans["complex"].append(uppercase_letters[i]+uppercase_letters[(i+1)%n])
        return ans

In [24]:
complexes={
    "C1": {
        "description": "Two triangles sharing an edge, one of them being hollow, the other filled",
        "complex": ["A", "B", "C", "D", "AB", "AC", "BC", "BD", "CD", "BCD"]
    },
    "C2": {
        "description": "Two hollow triangles sharing an edge",
        "complex": ["A", "B", "C", "D", "AB", "AC", "BC", "BD", "CD"]
    },
    "C3": {
        "description": "A Möbius strip",
        "complex": ["A","B","C","D","E","AB","AC","AD","AE","BC","BD","BE","CD","CE","ABC","ACD","ACE","BCD","BCE"]
    },
    "C5": {
        "description": "A hollow tetrahedron sharing an edge with a hollow triangle",
        "complex": ["A", "B", "C", "D", "E", "CE", "DE", "AB", "AC", "AD", "BC", "BD", "CD", "BCD", "ABC", "ABD", "ACD"]
    },
    "T2": {
        "description": "A triangulated 2-torus",
        "complex": ["A","B","C","D","E","F","G","AB","AC","AD","AE","AF","AG","BC","BD","BE","BF","BG","CD","CE","CF","CG","DE","DF","DG","EF","EG","FG","ABD","ABF","ACD","ACG","AEF","AEG","BCE","BCG","BDE","BFG","CDF","CEF","DEG","DFG"]
    },
    "Triangle":{"description":"A triangle","complex":["A","B","C","AB","AC","BC"]}

}

In [223]:
C=make_polygon(10)

In [224]:
C["description"]

'A polygon with 10 sides '

In [225]:
C["complex"]

['A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'AB',
 'BC',
 'CD',
 'DE',
 'EF',
 'FG',
 'GH',
 'HI',
 'IJ',
 'JA']

In [226]:
complex=C["complex"]
d=boundary(complex)

In [227]:
d

[array([[-1,  0,  0,  0,  0,  0,  0,  0,  0,  1],
        [ 1, -1,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  1, -1,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  1, -1,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  1, -1,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  1, -1,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  1, -1,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  1, -1,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  1, -1,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  1, -1]])]

In [228]:
D=compute_laplacians(d)

In [229]:
compute_harmonics(D)

[1, 1]

In [230]:
H=homology(d)
betti(H)

[1, 1]

### My experiment

In [235]:
def check_mod_2(M):
    d=M.shape[0]
    for i in range(d):
        print(M[i][i],sum([M[i][j] for j in range(d) if not i==j]))

In [236]:
check_mod_2(D[1])

2 -2
2 -2
2 -2
2 -2
2 -2
2 -2
2 -2
2 -2
2 -2
2 -2


In [248]:
def nonempty_subsequences(s):
    """
    Generate all nonempty subsequences of a given string that are not the whole string.

    Parameters:
    s (str): The input string.

    Returns:
    List[str]: A list of all nonempty subsequences not including the whole string.
    """
    subsequences = []

    def generate_subsequences(current, index):
        if index == len(s):
            if current and current != s:
                subsequences.append(current)
            return
        # Include the current character
        generate_subsequences(current + s[index], index + 1)
        # Exclude the current character
        generate_subsequences(current, index + 1)

    generate_subsequences("", 0)
    return sorted(subsequences,key=lambda z:len(z))


In [253]:
def make_sphere(d):
    """Makes a string of dimension d"""
    """Create a list of uppercase letters"""
    uppercase_letters = list(string.ascii_uppercase)
    if d+1>len(uppercase_letters):
        return "Not enough letters"
    else:
        ans=dict()
        ans["description"]=f"A sphere of dimension {d}"
        ans["complex"]=nonempty_subsequences(uppercase_letters[:d+1])[:-1]
        return ans
    

In [254]:
S=make_sphere(3)

In [255]:
S["complex"]

['A',
 'B',
 'C',
 'D',
 'AB',
 'AC',
 'AD',
 'BC',
 'BD',
 'CD',
 'ABC',
 'ABD',
 'ACD',
 'BCD']

In [247]:


# Example usage
input_string = "abc"
result = nonempty_subsequences(input_string)

print(result)

['a', 'b', 'c', 'ab', 'ac', 'bc']
